## Mice 3D Micro-CT Segmentation and Visualization

**Import necessary libraries**

In [1]:
import sys
sys.path.append("..") 
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
from pathlib import Path
import os
import imageio
import torch.cuda
import segmentation_models_pytorch as smp
from collections import defaultdict
import nibabel as nib
from tqdm import tqdm
from Utils.dataset_utils import *
from Utils.prediction_utils import *
from Utils.visualization_utils import *
from ipywidgets import *
from IPython.display import clear_output, display
from tkinter import Tk, filedialog
import SimpleITK as sitk

In [2]:
DEVICE = 'cuda'
MODEL_NAME='Unet'
ENCODER='efficientnet-b4'
ENCODER_WEIGHTS='imagenet'
BEST_WEIGHTS=r"G:\Documents\GitHub122121\MicroCTsegmentation\Mice CT Training weights - Final\Unet_efficientnet-b4\best_score.pt"

In [3]:
def select_files(b):
    clear_output()                                         # Button is deleted after it is clicked.
    root = Tk()
    root.withdraw()                                        # Hide the main window.
    root.call('wm', 'attributes', '.', '-topmost', True)   # Raise the root to the top of all windows.
    b.files = filedialog.askopenfilename(multiple=True)    # List of selected files will be set button's file attribute.
    print(b.files)  
def select_folder(b):
    clear_output() 
    root = Tk() # pointing root to Tk() to use it as Tk() in program.
    root.withdraw() # Hides small tkinter window.
    root.attributes('-topmost', True) # Opened windows will be active. above all windows despite of selection.
    b.folder = filedialog.askdirectory()
    print(b.folder)

**Select Model Weights**

In [4]:
weight_select = Button(description="Select Weights")
weight_select.on_click(select_files)
display(weight_select)

('G:/Documents/GitHub122121/MicroCTsegmentation/Mice CT Training weights - Final/Unet_efficientnet-b4/best_score.pt',)


In [6]:
BEST_WEIGHTS = weight_select.files[0]

**Load Model*

In [7]:
preprocessing_fn = smp.encoders.get_preprocessing_fn(ENCODER, ENCODER_WEIGHTS)
best_model = torch.load(BEST_WEIGHTS, map_location=torch.device(DEVICE))

C:\Users\ns3597\AppData\Local\Temp/ipykernel_68036/2587161121.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  best_model = torch.load(BEST_WEIGHTS, map_location=torch.de

**Select Subject**

In [8]:
subject_select = Button(description="Select Subject")
subject_select.on_click(select_files)
display(subject_select)

('G:/Data/microCT/rawDICOM/C57mice_02042021/C57-#9-MicroCT-020421 - Cardiac CT_105254-systole/Elaine Wan Cardio Test_362_Mouse 9_105254-systole_0512.nii',)


In [9]:
SUBJECT_PATH=subject_select.files[0]

In [10]:
SUBJECT=os.path.split(SUBJECT_PATH)[-1]

In [11]:
volume,gt_mask,affine=load_case(SUBJECT_PATH,"")

In [12]:
volume_pred_mask=predict_volume(best_model,volume,True,preprocessing_fn)
volume_pred_mask=np.round(volume_pred_mask)

100%|████████████████████████████████████████████████████████████████████████████████| 512/512 [00:31<00:00, 16.33it/s]


**Predict Volume**

**Saving predicted mask as .nii and .stl**

In [13]:
def create_folder(path):
    if os.path.exists(path)==False:
        os.mkdir(path)
save_folder=os.path.split(SUBJECT_PATH)[:-1][0]
create_folder(os.path.join(save_folder,'Output'))
create_folder(os.path.join(save_folder,'Movie'))

In [14]:
save_mask_nii(volume_pred_mask,affine,os.path.join(save_folder,'Output','prediction.nii.gz'))

**Display Results - 2D**

In [15]:
def display_2d(idx):
    visualize(
            Z=draw_mask(volume[idx,:,:],volume_pred_mask[idx,:,:]),
            Y=draw_mask(volume[:,idx,:],volume_pred_mask[:,idx,:]),
            X=draw_mask(volume[:,:,idx],volume_pred_mask[:,:,idx])
             )
interact(display_2d,idx=widgets.IntSlider(min=0, max=512, step=1, value=255))


interactive(children=(IntSlider(value=255, description='idx', max=512), Output()), _dom_classes=('widget-inter…

<function __main__.display_2d(idx)>

**Save GIF**

In [16]:
def make_gif(volume,volume_pred_mask=None):
    img_arr=[]
    for idx in tqdm(range(volume.shape[0])):
        Z=draw_mask(volume[idx,:,:],volume_pred_mask[idx,:,:])
        Y=draw_mask(volume[:,idx,:],volume_pred_mask[:,idx,:])
        X=draw_mask(volume[:,:,idx],volume_pred_mask[:,:,idx])
#         print(X.shape,Y.shape,Z.shape)
        stacked=np.hstack((X,Y,Z))
        img_arr.append(stacked)
    imageio.mimsave(os.path.join(save_folder,'Movie','movie.gif'), img_arr)
    del img_arr
make_gif(volume,volume_pred_mask)

100%|████████████████████████████████████████████████████████████████████████████████| 512/512 [00:15<00:00, 32.64it/s]


**Display Results - 3D**

In [17]:
import meshplot as mp
from skimage.measure import marching_cubes_lewiner
v1,f1,_,_ = marching_cubes_lewiner(volume_pred_mask)
p = mp.plot(v1, f1,c=v1[:,0], return_plot=True)

C:\Users\ns3597\AppData\Local\Temp/ipykernel_68036/3201479894.py:3: FutureWarning: marching_cubes_lewiner is deprecated in favor of marching_cubes. marching_cubes_lewiner will be removed in version 0.19
  v1,f1,_,_ = marching_cubes_lewiner(volume_pred_mask)
C:\Users\ns3597\AppData\Local\anaconda3\envs\condaenv\lib\site-packages\jupyter_client\session.py:716: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(261.5, 33…